**Step 1) Importing and Instantiating Classes**

In [ ]:
# pip install sweetviz

In [ ]:
from utils import open_csv_file, analyze_dataframe, create_report
from correlation_matrix import Correlation
from data import DataFrame

import numpy as np

from graph import Graph
import seaborn as sns

training_data = open_csv_file('train.csv')
data_frame_instance = DataFrame(training_data)

analysis_result = analyze_dataframe(data_frame_instance)
print("Analysis Result:")
print(analysis_result)

**Step 2) Inspecting and Cleaning Data**

In [ ]:
# Check missing data
# print(data_frame_instance.check_missing_data())
# Check data types
# print(data_frame_instance.check_data_type())

# Replace missing values with median
data_frame_instance.replace_missing_value_with_median('Age')
# print(data_frame_instance.get_df())

**Step 3) Transforming Data**

In [ ]:
# Create column for last name
data_frame_instance.create_last_name_column('Name')
# print(data_frame_instance.get_df()['last_name'])

In [ ]:
# Label encoding
data_frame_instance.label_encoding('Last_name')
# print(data_frame_instance.get_df())

# One-hot encoding
data_frame_instance.hot_encoding('Embarked')
data_frame_instance.hot_encoding('Sex')
# print(data_frame_instance.get_df()['Embarked_Q'])
# print(data_frame_instance.get_df()['Embarked_S'])
# print(data_frame_instance.get_df()['Embarked_C'])
# print(data_frame_instance.get_df()['Sex_male'])
# print(data_frame_instance.get_df()['Sex_female'])

In [ ]:
# graph_instance = Graph(sns)
# graph_instance.bar_graph(data_frame_instance.get_df()['Last_name_encoded'])

In [ ]:
# print(data_frame_instance.get_df()['Last_name_encoded'])

In [ ]:
data_frame_instance.min_max_scaler('Last_name_encoded')
# print(data_frame_instance.get_df()['Last_name_encoded_scaled'])

In [ ]:
data_frame_instance.get_bin_edges('Fare','quantile')

In [ ]:
data_frame_instance.bucket_quantile('Age')
data_frame_instance.bucket_quantile('Fare')

In [ ]:
data_frame_instance.remove_columns(['PassengerId', 'Name', 'Cabin', 'Age', 'Fare', 'Ticket', 'Last_name'])

In [ ]:
# print(data_frame_instance.get_df())

**Step 4) Feature Selection**

In [ ]:
# ## feature selection
# corr_mat_type_two = data_frame_instance.get_df().corr().abs()
# ## created a matrix of ones with the same shape with corr mat
# mask = np.triu(np.ones_like(corr_mat_type_two, dtype=bool))
# mask_df = corr_mat_type_two.mask(mask)
# mask_df

In [ ]:
# correlation_instance = Correlation(data_frame_instance.get_df())
# correlation_instance.create_correlation_matrix()

In [ ]:
# correlation_instance.filter_most_correlated_features()

**Machine learning**